In [1]:
from pymongo import MongoClient
import pandas as pd
import re


In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [3]:
datab = db.companies.find()
df = pd.DataFrame(datab)
display(df.head(1)) # DataFrame original
df.shape # Comprobar filas y columnas del DF original

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]


(18801, 42)

In [4]:
# Se empieza a limpiar el DF con queries por pymongo

In [5]:
mongo_office= db.companies.find({
    "offices":{"$not":{"$size":0}}},
    {"name":1, "offices":1, "category_code":1, "founded_year":1,"description":1,
     "deadpooled_year":1, "number_of_employees":1, "total_money_raised":1,}) 
# Empresas con una o más oficinas

In [6]:
df_office = pd.DataFrame(mongo_office)


In [7]:
df_office.isna().sum()

_id                        0
category_code            652
deadpooled_year        12957
description             3454
founded_year            2487
name                       0
number_of_employees     6277
offices                    0
total_money_raised         0
dtype: int64

In [8]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


new_df_coords = df_office[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [9]:
def get_country_code(data):
    data = data['offices']
    if data[0]['country_code']:
        return data[0]

In [10]:
new_country_code = df_office[["offices"]].apply(get_country_code, result_type="expand", axis=1)

In [11]:
# concatenar dataframes 
df_clean = pd.concat([new_df_coords,df_office, new_country_code], axis=1)[["name","category_code", "deadpooled_year", 
                                                            "founded_year","description","oficina_principal", "lat", 
                                                         "lng", "number_of_employees", "total_money_raised", "country_code"]]




In [12]:
df_clean.isna().sum()

name                       0
category_code            652
deadpooled_year        12957
founded_year            2487
description             3454
description             1779
oficina_principal       4126
lat                     4126
lng                     4126
number_of_employees     6277
total_money_raised         0
country_code               0
dtype: int64

In [13]:
df_clean.head()

,name,category_code,deadpooled_year,founded_year,description,description,oficina_principal,lat,lng,number_of_employees,total_money_raised,country_code
0,AdventNet,enterprise,2.0,1996.0,Server Management Software,Headquarters,"{'type': 'Point', 'coordinates': [-121.904945,...",37.692934,-121.904945,600.0,$0,USA
1,Zoho,software,3.0,2005.0,Online Business Apps Suite,Headquarters,"{'type': 'Point', 'coordinates': [-121.904945,...",37.692934,-121.904945,1600.0,$0,USA
2,Digg,news,NaN,2004.0,user driven social content website,None,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523,60.0,$45M,USA
3,Postini,web,NaN,1999.0,None,None,"{'type': 'Point', 'coordinates': [-122.247573,...",37.506885,-122.247573,NaN,$0,USA
4,Geni,web,NaN,2006.0,Geneology social network site,Headquarters,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,18.0,$16.5M,USA


In [14]:
#limpiar nulls de lat y lng
df_clean.dropna(subset=["lat", "lng"], inplace=True)

In [15]:
df_clean.isna().sum()

name                      0
category_code           218
deadpooled_year        8983
founded_year           1654
description            2703
description            1532
oficina_principal         0
lat                       0
lng                       0
number_of_employees    4004
total_money_raised        0
country_code              0
dtype: int64

In [16]:
# Limpiando todas las empresas que han cerrado (deadpooled)
df_clean = df_clean[df_clean['deadpooled_year'].isna()]

In [17]:
# Quitando todos los nulos de number_of_employees
df_clean.dropna(subset=["number_of_employees"], inplace=True)

# Limpiando todas las empresas que tienen 0 trabajadores
df_clean = df_clean.drop(df_clean[df_clean["number_of_employees"]== 0].index)

In [18]:
df_clean['founded_year']=df_clean['founded_year'].fillna(0.0)

In [19]:
df_clean['founded_year'].dtype

dtype('float64')

In [20]:
# Cambiando los nulls de la columna founded_year por 0
#df_clean['founded_year'].fillna(0, inplace=True)

# Pasando de float --> integer la columna founded_year
df_clean['founded_year']=df_clean['founded_year'].astype(int)
df_clean['number_of_employees']=df_clean['number_of_employees'].astype(int)

In [21]:
# Limpiando los nullos de category_code
df_clean.dropna(subset=["category_code"], inplace=True)

In [22]:
# Recuento de valores para categorizar 
df_clean['category_code'].value_counts()

web                 1130
software             977
advertising          374
other                325
games_video          317
mobile               284
enterprise           272
consulting           267
ecommerce            243
network_hosting      209
public_relations     195
search               143
hardware              94
security              43
analytics             37
cleantech             35
social                32
biotech               25
education             20
finance               19
music                 17
travel                17
semiconductor         15
news                  14
messaging             12
photo_video            8
health                 8
real_estate            7
legal                  5
fashion                4
hospitality            4
sports                 4
medical                3
transportation         3
design                 2
automotive             2
nanotech               1
nonprofit              1
manufacturing          1
Name: category_code, dtyp

In [23]:
def re_category_code(x):
    #Categorizando en empresas tecnólogicas y en empresas de videjuegos
    x = str(x)
    if (re.search ('\s*([Gg]ame)\s*', x)):
        return 'Empresa de videojuegos'
    elif (re.search ('\s*(web)|(ware)|(tech)|(mobile)|(network_hosting)|(messaging)\s*', x)):
        return 'Empresa tecnologica'
    elif (re.search ('\s*\s*', x)):
        return 'None'

In [24]:
df_clean['category_code']=df_clean['category_code'].apply(re_category_code)

#Comprobando con cuantos valores me quedo
df_clean['category_code'].value_counts()

Empresa tecnologica       2767
None                      2085
Empresa de videojuegos     317
Name: category_code, dtype: int64

In [25]:
df_clean['category_code'].value_counts()

Empresa tecnologica       2767
None                      2085
Empresa de videojuegos     317
Name: category_code, dtype: int64

In [26]:
# Limpiando las columnas con 0 total_money_raised
df_clean=df_clean[df_clean.total_money_raised!='$0']
df_clean=df_clean[df_clean.total_money_raised!='€0']
df_clean=df_clean[df_clean.total_money_raised!='¥0']
df_clean=df_clean[df_clean.total_money_raised!='£0']

df_clean.head()

    

,name,category_code,deadpooled_year,founded_year,description,description,oficina_principal,lat,lng,number_of_employees,total_money_raised,country_code
2,Digg,None,NaN,2004,user driven social content website,None,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523,60,$45M,USA
4,Geni,Empresa tecnologica,NaN,2006,Geneology social network site,Headquarters,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,18,$16.5M,USA
12,Scribd,None,NaN,2007,Read Unlimited Books,HQ,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,50,$25.8M,USA
15,Twitter,None,NaN,2006,Real time communication platform,,"{'type': 'Point', 'coordinates': [-122.4169244...",37.776805,-122.416924,1300,$1.16B,USA
16,Facebook,None,NaN,2004,Social network,Headquarters,"{'type': 'Point', 'coordinates': [-122.151801,...",37.416050,-122.151801,5299,$2.43B,USA


In [27]:
df_clean.isna().sum()

name                      0
category_code             0
deadpooled_year        1466
founded_year              0
description             103
description             161
oficina_principal         0
lat                       0
lng                       0
number_of_employees       0
total_money_raised        0
country_code              0
dtype: int64

In [28]:
df_clean['category_code'].value_counts()


Empresa tecnologica       746
None                      618
Empresa de videojuegos    102
Name: category_code, dtype: int64

In [29]:
df_clean = df_clean.drop("description", axis=1)

In [30]:
df_clean

,name,category_code,deadpooled_year,founded_year,oficina_principal,lat,lng,number_of_employees,total_money_raised,country_code
2,Digg,None,NaN,2004,"{'type': 'Point', 'coordinates': [-122.394523,...",37.764726,-122.394523,60,$45M,USA
4,Geni,Empresa tecnologica,NaN,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,18,$16.5M,USA
12,Scribd,None,NaN,2007,"{'type': 'Point', 'coordinates': [-122.404052,...",37.789634,-122.404052,50,$25.8M,USA
15,Twitter,None,NaN,2006,"{'type': 'Point', 'coordinates': [-122.4169244...",37.776805,-122.416924,1300,$1.16B,USA
16,Facebook,None,NaN,2004,"{'type': 'Point', 'coordinates': [-122.151801,...",37.416050,-122.151801,5299,$2.43B,USA
21,Plaxo,Empresa tecnologica,NaN,2002,"{'type': 'Point', 'coordinates': [-122.055197,...",37.387845,-122.055197,50,$28.3M,USA
22,MeetMoi,None,NaN,2007,"{'type': 'Point', 'coordinates': [-73.985506, ...",40.757929,-73.985506,15,$5.58M,USA
23,eBay,Empresa tecnologica,NaN,1995,"{'type': 'Point', 'coordinates': [-121.930035,...",37.295005,-121.930035,15000,$6.7M,USA
24,Powerset,None,NaN,2006,"{'type': 'Point', 'coordinates': [-122.395289,...",37.778613,-122.395289,60,$22.5M,USA
25,Technorati,None,NaN,2002,"{'type': 'Point', 'coordinates': [-122.393041,...",37.779558,-122.393041,35,$32.1M,USA


In [31]:
df_clean=df_clean[df_clean["category_code"]!='None']


df_clean

,name,category_code,deadpooled_year,founded_year,oficina_principal,lat,lng,number_of_employees,total_money_raised,country_code
4,Geni,Empresa tecnologica,NaN,2006,"{'type': 'Point', 'coordinates': [-118.393064,...",34.090368,-118.393064,18,$16.5M,USA
21,Plaxo,Empresa tecnologica,NaN,2002,"{'type': 'Point', 'coordinates': [-122.055197,...",37.387845,-122.055197,50,$28.3M,USA
23,eBay,Empresa tecnologica,NaN,1995,"{'type': 'Point', 'coordinates': [-121.930035,...",37.295005,-121.930035,15000,$6.7M,USA
30,Mahalo,Empresa tecnologica,NaN,2007,"{'type': 'Point', 'coordinates': [-118.487267,...",34.017606,-118.487267,40,$21M,USA
32,Kyte,Empresa de videojuegos,NaN,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,40,$23.4M,USA
34,Jingle Networks,Empresa tecnologica,NaN,2005,"{'type': 'Point', 'coordinates': [-122.173887,...",37.480999,-122.173887,35,$88.7M,USA
37,Meetup,Empresa tecnologica,NaN,2002,"{'type': 'Point', 'coordinates': [-73.995722, ...",40.726040,-73.995722,75,$18.3M,USA
42,Cisco,Empresa tecnologica,NaN,1984,"{'type': 'Point', 'coordinates': [-121.95377, ...",37.408802,-121.953770,63000,$2.5M,USA
47,Yahoo!,Empresa tecnologica,NaN,1994,"{'type': 'Point', 'coordinates': [-122.025485,...",37.418531,-122.025485,13600,$6.8M,USA
48,Jajah,Empresa tecnologica,NaN,2005,"{'type': 'Point', 'coordinates': [-122.0899512...",37.423390,-122.089951,110,$33M,USA


In [33]:
df_clean.to_json('/home/jjespper/Escritorio/proyecto_final1.json', orient='records')
